In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [2]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample.csv')
benign_df.head(2)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,15.854422,-0.343690,-0.086569,0.096351,0.142000,0.002723,0.706619,1.982467,1.798333,-0.034669,...,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336,1
1,-0.060352,5.598377,2.253048,4.681794,3.829148,0.125465,3.569438,-0.039001,2.481753,5.125594,...,-0.059807,1.103561,0.162506,0.515708,1.673959,6.184780,12.942534,8.511326,2.479016,1


In [5]:
udp_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\UDP_resample_4000.csv')
udp_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.025978,-0.119415,-0.495813,-0.005438,-0.515642,-0.002646,-0.423139,0.371911,0.083769,-0.951297,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
1,0.025978,0.019019,0.059450,-0.005438,0.019607,-0.002646,-0.616152,-0.670846,-0.653085,0.440479,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
2,0.025978,0.162140,0.614713,-0.005438,0.589549,-0.002646,0.155901,-0.879398,-0.696430,1.217554,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
3,0.025978,0.018732,0.059450,-0.005438,0.019607,-0.002646,-0.616152,-0.670846,-0.653085,0.440479,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
4,0.025978,0.162946,0.614713,-0.005438,0.589549,-0.002646,0.155901,-0.879398,-0.696430,1.217554,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,0.025978,-0.119415,-0.495813,-0.005438,-0.502426,-0.002646,-0.165788,0.557290,0.314939,-0.951297,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
3675,0.025978,0.018761,0.059450,-0.005438,0.052647,-0.002646,0.027225,-0.670846,-0.364123,1.154210,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
3676,0.025978,-0.119415,-0.495813,-0.005438,-0.472690,-0.002646,0.413252,0.974393,0.835071,-0.951297,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1
3677,0.025978,-0.119410,-0.495813,-0.005438,-0.472690,-0.002646,0.413252,0.974393,0.835071,-0.951297,...,0.151105,-0.004773,-0.003397,-0.005162,-0.004036,-0.007978,-0.002093,-0.007954,-0.007811,-1


In [3]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_4000.csv')

In [8]:
# novelty detection
df_test = pd.concat([benign_test_df, udp_df], ignore_index=True)
df_train = benign_df

In [9]:
df_test = df_test[[' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length',' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' min_seg_size_forward', ' Label']]
df_train = df_train[[' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length',' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' min_seg_size_forward', ' Label']]

In [10]:
# training_data
X_train = df_train.drop([' Label'], axis=1)
y_train = df_train[' Label']
# testing_data
X_test = df_test.drop([' Label'], axis=1)
y_test = df_test[' Label']

In [11]:
# Novelty detection with 8000 sample balanced
for a in range(1, 5):
    for i in arange(0.1, 0.5, 0.1):
        clf = LocalOutlierFactor(n_neighbors=a, novelty=True, contamination=i)
        clf.fit(X_train)
        y_pred_test = clf.predict(X_test)
        print('Neighbour : ' + str(a))
        print('contamination : ' + str(i))
        print('---Classification Report---')
        print(classification_report(y_test, y_pred_test))

Neighbour : 1
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3679
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      7682
   macro avg       1.00      1.00      1.00      7682
weighted avg       1.00      1.00      1.00      7682

Neighbour : 1
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3679
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      7682
   macro avg       1.00      1.00      1.00      7682
weighted avg       1.00      1.00      1.00      7682

Neighbour : 1
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3679
           1       1.00      1.00  